# Chapter 9: Subqueries

As seen in Chapter 3, subqueries are quite powerful tools to filter conditions.

Let's explore that some more.


In [1]:
import os

from dotenv import load_dotenv
from sqlalchemy import create_engine, URL, select, func
from sqlalchemy.orm import Session
import pandas as pd

from utils import print_sql_statement


load_dotenv()

url_object = URL.create(
    os.environ["DB_ENGINE"],
    username=os.environ["DB_USER"],
    password=os.environ["DB_PASSWD"],
    host=os.environ["DB_HOST"],
    database=os.environ["DB_NAME"],
)

engine = create_engine(url_object)

# Non-correlated, Multirow Subqueries

For simple queries like find the employees who are superiors, would be written like so using the IN operator

```sql
SELECT
    emp_id
    , fname
    , lname
    , title
FROM employee
WHERE emp_id IN (
    SELECT superior_emp_id
    FROM employee
)
```

or the less intuitive ANY/ALL operator


```sql
SELECT
    emp_id
    , fname
    , lname
    , title
FROM employee
WHERE emp_id = ANY (
    SELECT superior_emp_id
    FROM employee
)
```

However, the ANY/ALL operator would be more intuitive in this case: Find all accounts having an available balance smaller than all of Frank Tucker's accounts.

In [12]:
from sqlalchemy import and_

from model import Account, Individual


with Session(engine) as session:
    df = pd.read_sql_query(
        """
        SELECT
            account_id
            , avail_balance
        FROM account
        WHERE avail_balance < ALL (
            SELECT
                avail_balance
            FROM account a
            JOIN individual ind ON a.cust_id = ind.cust_id
            WHERE ind.fname = 'Frank' AND ind.lname = 'Tucker'
        )
        ORDER BY account_id;
        """,
        con=session.connection()
    )
    target_individual: Individual = (
        session.query(Individual)
        .where(
            and_(
                Individual.fname == "Frank",
                Individual.lname == "Tucker"
            )
        )
        .scalar()
    )
    target_accounts: list[Account] = (
        session.query(Account)
        .where(Account.cust_id == target_individual.cust_id)
        .all()
    )
    results = sorted(
        (
            (acct.account_id, acct.avail_balance)
            for acct in session.query(Account)
            if all(
                acct.avail_balance < tgt_acct.avail_balance
                for tgt_acct in target_accounts
            )
        ),
        key=lambda tup: tup[0]
    )

print(df)
print(results)

   account_id  avail_balance
0           2         500.00
1           5         200.00
2          10         534.12
3          11         767.77
4          14         122.37
5          19         387.99
6          21         125.67
7          25           0.00
[(2, 500.0), (5, 200.0), (10, 534.12), (11, 767.77), (14, 122.37), (19, 387.99), (21, 125.67), (25, 0.0)]
